# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298349378816                   -0.85    5.0
  2   -8.300193002932       -2.73       -1.26    1.0
  3   -8.300443617864       -3.60       -1.89    3.5
  4   -8.300462198615       -4.73       -2.77    1.8
  5   -8.300464187298       -5.70       -3.06   10.4
  6   -8.300464406028       -6.66       -3.24    1.0
  7   -8.300464523502       -6.93       -3.38    2.1
  8   -8.300464587134       -7.20       -3.52    1.0
  9   -8.300464627096       -7.40       -3.70    1.0
 10   -8.300464639483       -7.91       -3.88    2.1
 11   -8.300464643789       -8.37       -4.34    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67467077881                   -0.70    6.1
  2   -16.67872283454       -2.39       -1.14    2.0
  3   -16.67923424250       -3.29       -1.86    3.4
  4   -16.67927953984       -4.34       -2.77    3.8
  5   -16.67928607035       -5.19       -3.23    5.5
  6   -16.67928621364       -6.84       -3.50    3.1
  7   -16.67928622116       -8.12       -3.96    1.0
  8   -16.67928622407       -8.54       -4.63    1.1


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.29659392131                   -0.56    7.0
  2   -33.33280765003       -1.44       -1.00    1.6
  3   -33.33413090129       -2.88       -1.74    4.6
  4   -33.33692849349       -2.55       -2.28    6.9
  5   -33.33694159072       -4.88       -2.55    9.1
  6   -33.33694381479       -5.65       -3.46    2.2
  7   -33.33694391467       -7.00       -4.21    2.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298357992471                   -0.85    5.2
  2   -8.300239094528       -2.73       -1.59    1.0
  3   -8.300448870856       -3.68       -2.55    4.8
  4   -8.300377610393   +   -4.15       -2.29   12.4
  5   -8.300464133842       -4.06       -3.44    5.1
  6   -8.300464575082       -6.36       -3.88    2.2
  7   -8.300464642655       -7.17       -4.74    2.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32683088223                   -0.56    6.4
  2   -33.32465269337   +   -2.66       -1.27    1.2
  3   -20.64038235136   +    1.10       -0.51    7.6
  4   -33.30732894696        1.10       -1.79    7.2
  5   -33.25851585839   +   -1.31       -1.50    3.2
  6   -32.95725096136   +   -0.52       -1.27    4.5
  7   -33.32272228511       -0.44       -1.87    5.0
  8   -33.33533011193       -1.90       -2.41    1.9
  9   -33.33642471755       -2.96       -2.58    3.4
 10   -33.33607218420   +   -3.45       -2.56    2.2
 11   -33.33666656757       -3.23       -2.76    1.8
 12   -33.33687768604       -3.68       -3.08    2.6
 13   -33.33694194671       -4.19       -3.49    3.2
 14   -33.33694309215       -5.94       -3.76    3.4
 15   -33.33694321260       -6.92       -4.07    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.